<a href="https://colab.research.google.com/github/hyoh79/ML/blob/main/USL_NMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#비음수 행렬분해(NMF: Non-negative Matrix Factorization)
!pip install mglearn
import mglearn

# ex1) Toy dataset
mglearn.plots.plot_nmf_illustration()
# 인위적 toy dataset에 NMF적용하기
# PCA와 달리 NMF로 데이터를 다루려면 주어진 데이터가 양수인지 확인해야함
# 데이터가 원점 (0,0)에서 상대적으로 어디에 놓여있는지가 중요함.
# 원점 (0,0)에서 데이터로 가는 방향을 추출하는 것, 즉 음수 미포함 성분을 이해할 수 있음.

# NMF로 데이터셋 표현하기 - 찾은 성분이 2개일 때 (왼쪽)와 1개일 때 (오른쪽)
# 1) 두 개의 성분으로 표현한 경우 - NMF는 데이터셋의 모든 포인트를 양수로 이뤄진 두 개의 성분으로 표현(왼쪽 그림)
# 2) 한 개의 성분으로 표현한 경우 - NMF는 데이터를 가장 잘 표현할 수 있는 평균으로 향하는 성분을 만듬.(오른쪽 그림)
# PCA와 NMF의 차이점
# 1) PCA와 달리 성분 개수를 줄이면 특정 방향이 제거되는 것뿐만 아니라 전체 성분이 완전히 바뀜!
# 2) PCA와 달리 NMF에서 성분은 특정 방식으로 정렬되어 있지 않아서 모든 성분을 동등하게 취급함.
# 3) NMF는 무작위로 초기화하기 때문에 난수 생성 초깃값에 따라 결과가 달라짐 ---> 복잡한 예제에서는 큰 차이를 만들 수 있음.

In [ ]:
# ex2) LFW dataset with NMF
# NMF의 핵심 매개변수는 추출할 성분의 개수. NMF의 성분의 개수 < 특성의 개수
# NMF를 사용해서 데이터를 재구성함에 있어 성분의 개수가 어떤 영향을 주는지 분석
from sklearn.datasets import fetch_lfw_people
import numpy as np
from sklearn.model_selection import train_test_split
people=fetch_lfw_people(min_faces_per_person=20, resize=0.7)
image_shape=people.images[0].shape

mask=np.zeros(people.target.shape,dtype=np.bool)
for target in np.unique(people.target):
  mask[np.where(people.target==target)[0][:50]]=1

X_people=people.data[mask]
y_people=people.target[mask]
X_people=X_people/255.

X_train,X_test,y_train,y_test=train_test_split(X_people,y_people,stratify=y_people,random_state=0)
mglearn.plots.plot_nmf_faces(X_train, X_test,image_shape)

# 변환을 되돌린 결과는 PCA를 사용했을 때와 비슷하지만 품질이 조금 떨어짐.
# PCA가 재구성 측면에서 최선의 방향을 찾기 때문
# NMF는 데이터를 인코딩하거나 재구성하는 용도로 사용하기 보다는 주로 데이터에 있는 유용한 패턴을 찾는 데 활용됨.

In [ ]:
# LFW dataset에서 NMF를 사용하여 15개의 성분만 추출
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt

nmf=NMF(n_components=15,random_state=0,max_iter=1000,tol=1e-2)
nmf.fit(X_train)
X_train_nmf=nmf.transform(X_train)
X_test_nmf=nmf.transform(X_test)

fig,axes=plt.subplots(3,5,figsize=(15,12),subplot_kw={'xticks':(),'yticks':()})

for i, (component,ax) in enumerate(zip(nmf.components_,axes.ravel())):
  ax.imshow(component.reshape(image_shape))
  ax.set_title("Component {}".format(i))

# NMF의 성분들은 모두 양수값이라서 PCA성분들보다 월씬 더 얼굴 원형처럼 보임.
# 예) 성분3: 오른쪽으로 조금 돌아간 얼굴로 보임
# 예) 성분7: 왼쪽으로 조금 회전한 얼굴로 보임

In [ ]:
# 결과, 성분3,7이 각각 특별히 강하게 나타난 이미지들을 시각화
# 예) 성분 3의 계수가 큰 얼굴들

compn=3
# 0번째 성분부터 시작하기 때문에 4번째 성분으로 정렬하여 처음 10개 이미지를 출력
inds=np.argsort(X_train_nmf[:,compn])[::-1]
fig,axes=plt.subplots(2,5,figsize=(15,8),subplot_kw={'xticks':(),'yticks':()})

for i, (ind,ax) in enumerate(zip(inds,axes.ravel())):
  ax.imshow(X_train[ind].reshape(image_shape))


# 예) 성분 7의 계수가 큰 얼굴들
# 예상데로 성분 7에 대한 계수 값이 큰 얼굴들은 왼쪽으로 돌아가 있음.
# 이와 같은 패턴을 추출하는 것은 "소리", "유전자 표현", "텍스트 데이터"처럼 덧붙이는 구조를 가진 데이터에 적합함.
compn=7
# 0번째 성분부터 시작하기 때문에 8번째 성분으로 정렬하여 처음 10개 이미지를 출력
inds=np.argsort(X_train_nmf[:,compn])[::-1]
fig,axes=plt.subplots(2,5,figsize=(15,8),subplot_kw={'xticks':(),'yticks':()})

for i, (ind,ax) in enumerate(zip(inds,axes.ravel())):
  ax.imshow(X_train[ind].reshape(image_shape))

In [ ]:
# ex3) Signal dataset with NMF
# 3개의 서로 다른 입력으로부터 합성된 신호 데이터셋 고려
S=mglearn.dataset.make_signals()
plt.figure(figsize=(6,1))
plt.plot(S,'-')
plt.xlabel("Time")
plt.ylable("Signal")

# 가정1: 원본 신호는 볼 수 없고 3개의 신호가 섞인 신호들만 관찰할 수 있는 상황.
# 가정2: 100개의 측정 장치로 섞인 신호들을 관찰할 수 있음.
# 결과: 섞인 신호를 분해해서 원본 신호를 복원해야함.
# 가정1,2를 고려해서 원본 데이터를 사용해 100개의 측정 데이터를 생성
A=np.random.RandomState(0).uniform(size=(100,3))
X=np.dot(S,A.T)
print("측정 데이터 형태:", X.shape)




# NMF를 사용해서 3개의